## MCP server

In [ ]:
from agents.mcp import MCPServerStdio

fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    fetch_tools = await server.list_tools()

fetch_tools

In [ ]:
from agents.mcp import MCPServerStdio

fetch_params = {"command": "uvx", "args": ["pubmedmcp@latest"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    pubmed_tools = await server.list_tools()

pubmed_tools

In [ ]:
from dotenv import load_dotenv
from agents import Runner, trace, function_tool, Agent, OpenAIChatCompletionsModel, SQLiteSession
from openai import AsyncOpenAI
from agents.mcp import MCPServerStdio


load_dotenv(override=True)


agent_instructions = """You are a helpful assistant for Bioinformatics research. 
You are equipped with pubmed mcp server. You can help users find relevant scientific articles, summarize research papers, and provide insights on various bioinformatics topics.
"""

# Define LLM model

# model = "gpt-4.1-mini"

# Define the OpenAI client pointing to a local LLM server
client = AsyncOpenAI(base_url="http://localhost:11434/v1")

# Define model using the local LLM server client
# use gpt-oss which is better for tool usage and reasoning even though it takes time
model = OpenAIChatCompletionsModel(model = "gpt-oss",openai_client= client)



# Define a session to maintain context across interactions
session = SQLiteSession("pubmed_agent_session.db")


fetch_params = {"command": "uvx", "args": ["pubmedmcp@latest"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    pubmed_tools = await server.list_tools()

    bioinfo_agent = Agent(name="BioinformaticsAgent",
                    instructions=agent_instructions,
                    model=model,
                    mcp_servers=[server])
    
    #query = "Find recent articles on CRISPR gene editing in plants and summarize the key findings."
    query="Find 5 review articles on latest use of AI in genomics. Provide urls of the articles in markdown format."
    
    result = await Runner.run(bioinfo_agent, query, session=session)
    print(result.final_output)


In [11]:
async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    pubmed_tools = await server.list_tools()

    bioinfo_agent = Agent(name="BioinformaticsAgent",
                    instructions=agent_instructions,
                    model=model,
                    mcp_servers=[server])
    
    async def chat(message, history):
        result = await Runner.run(bioinfo_agent, message, session=session)
        return result.final_output

    import gradio as gr
    gr.ChatInterface(
        chat,
        title="Clinical Expert Chatbot",
        description="Chat with the Clinical Expert"
    ).launch()



* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/mnt/c/Users/PriyabrataPanigrahi/Downloads/ai/aienv/lib/python3.12/site-packages/gradio/queueing.py", line 763, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/c/Users/PriyabrataPanigrahi/Downloads/ai/aienv/lib/python3.12/site-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/c/Users/PriyabrataPanigrahi/Downloads/ai/aienv/lib/python3.12/site-packages/gradio/blocks.py", line 2106, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/c/Users/PriyabrataPanigrahi/Downloads/ai/aienv/lib/python3.12/site-packages/gradio/blocks.py", line 1586, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/c/Users/PriyabrataPanigrahi/Downloa